전처리 및 LDA 토픽 모델링_코드

본 코드는 크롤링한 뉴스 텍스트 데이터를 가지고 조사나 TF-IDF 점수를 통해 불필요한 불용어를 제거하여 정치적 올바름 키워드를 중심으로 한 뉴스 기사들의 단어빈도를 파악 후 , LDA 모델을 생성하는데 적정한 토픽 수를 정하였다. 토픽 수를 정하는데는 혼잡도와 복잡도를 시각화하여 확인해보았으나 유의미한 결과를 발견하지 못하여 pyLDAVis 시각화를 활용하여 토픽 별 독립성이 가장 높게 나타나는 토픽 수를 3을 선정하여 토픽모델링을 진행하였다. 토픽모델링 진행 결과, 토픽은 "정치적 다양성과 공정성 담론", "전통 정치 담론과 정당 갈등", " 미국 보수 포퓰리즘과 정체성 정치의 충돌" 으로 묶어볼 수 있었다. 토픽 순으로 2117, 1080, 934개의 뉴스 문서의 개수를 추출할 수 있었으며 토픽별 단어 포함 확률인 β와 문서의 토픽 분류 확률인 γ를 defalutdict로 추출하여 가장 관련된 문서들을 확인하였다. 연도별 토픽의 수를 확인하여 막대그래프로 시각화를 하였다. 

In [ ]:
import pandas as pd
import re
from konlpy.tag import Okt

# 1. CSV 불러오기
merged_df = pd.read_csv('merged_df.csv', dtype=str)

# 2. 제목과 본문 합치기
merged_df['text'] = merged_df['제목'].fillna('') + ' ' + merged_df['본문'].fillna('')

# 3. 영문자, 숫자, 특수기호 제거 (한글 및 공백만 남김)
def clean_non_korean(text):
    text = re.sub(r'[^가-힣\s]', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

merged_df['clean_text'] = merged_df['text'].apply(clean_non_korean)

# 4. 형태소 분석기 초기화
okt = Okt()

# 5. 명사만 추출
merged_df['nouns'] = merged_df['clean_text'].apply(lambda x: okt.nouns(x))

# 6. 불용어 리스트 정의 (예시)
domain_stopwords = set([
    '뉴스', '기자', '기사', '보도', '언론', '인터뷰',
    '발표', '전달', '보도자료', '관련', '오늘', '이번',
    '좀','정치적', '수', '있다', '이', '대한', '있는', '한', '년', '등', '더',
    '그', '것이다', '같은', '기자', '다른', '것은', '일', '전', '했다', '하는',
    '미국', '이런', '위해', '하지만', '할', '지난', '월', '또', '것이', '대해',
    '한다', '가장', '없는', '있다고', '것을', '그는', '및', '통해', '아니라',
    '있다는', '때', '것', '많은', '중', '게', '최근', '모든', '큰', '어떤',
    '무단' ,'배포' ,'전', '행사' ,'금지','정치적', '수', '있다', '이', '대한',
    '있는', '한', '년', '등', '더', '그', '것이다', '같은', '기자', '다른', '것은', 
    '일', '전', '했다', '하는', '미국', '이런', '위해', '하지만', '할', '지난', '월',
    '또', '것이', '대해', '한다', '없는', '가장', '있다고', '것을', '그는', '및', '통해',
    '아니라', '있다는', '때', '것', '많은', '중', '게', '최근', '모든', '큰', '어떤',
    '것으로', '이후', '함께', '때문에', '없다', '자신의', '될', '두', '그러나', '않는', '말했다', 
    '다시', '안', '우리', '특히', '하고', '된', '것이라고', '위한', '그런', '건', '아니다', '때문이다', '내', 
    '모두', '어떻게', '잘', '그리고', '이번', '그의', '지금', '왜', '되는', 
    '많이', '보면', '그런데', '대', '우리가', '이렇게', '그래서', '있습니다', '저는',
    '는','다','역시','않은','나는','이제','여러','이를','내가','등을','당시','제가',
    '의','않고','가','데','것도','한다고','있을','된다','물론','한다는','바로','사실',
    '이미','에','그렇게','좋은','있고','말','사람','거','점','재','속','명','책','시작',
    '힘','하나','교수','시간','문화','표현','이유','생각','문제','한국','이야기','대표','세계',
    '후보','주장','상황','주의','작품','자신','정도','위','국가','모습','얘기','시대','뒤','못',
    '가지','캐릭터','의미','경우','분','의원','발언','운동','결과','과정','그것','현실','작가','만','사건',
    '평가','배우','를','등장','가치','경제','저','인간','걸','글','날','감독','입장','기업','존재','부분',
    '무엇','앞','세상','자기','공개','주인공','개인','현재','강조','당','삶','계속','마음','방송','자체',
    '방식','결정','인물','선택','과거','내용','분석','나라','개','사이','중심','일부','정당','때문','알','후',
    '나','번','관객','년대','여기','사용','설명','처음','아이','뜻','선','미래','이름','지난해','보고','뿐','세',
    '정의','적','시민','대상','자리','지역','볼','비','돈','온','곳','정치인','해','쪽','역할','사랑','권력','향',
    '공격','관계','감','누구','달','대중','가능성','요구','라며','수도','올해','간','목소리','진행','정체','위기',
    '개봉','표','이자','관심','살','오히려','층','질문','콘텐츠','정권','로','활동','시리즈','언어','행동','의견',
    '온라인','장면','또한','진영','최고','달러','영향','세력','제','이슈','전략','이기','고민','첫','현상','듯'
    ,'저자', '인기', '기존', '스스로', '주', '은', '집단', '바', '경험', '국제', '노력', '부정', 
    '기준', '국내', '동안', '여러분', '장관', '도널드', '줄', '학교', '로서', '논쟁', '요',
    '기록', '관', '도', '눈', '기도', '반응','대신', '조사', '참여', '라면', '해결', '메시지', '차', 
    '거나', '비롯', '필요', '영상', '다음', '약', '시절', '제기', '서로', '의식', '선언',
    '책임', '론', '단어', '포함', '직접', '편', '다만', '크게', '주목', '정말', 
    '지지율', '가운데', '몇', '우려', '김', '해당', '회사', '시', '언급', '사태', '출연', '독자',
    '말씀', '인사', '전체', '인정', '더욱', '입', '공식', '태도', '기회', '저작권', 
    '답', '해도', '방법', '단체', '기술', '박', '상대', '사진', '조금', '제작', '당신', '해석', '핵심', '동시', '회', 
    '예', '거부', '제도', '일이', '집', '사례', '매우', '음악', '여론조사', '역', '연기', '팀','실제','이해','추구','정신',
    '제대로','대부분','길','이미지','주요','리','앞서','거의','마지막','매체','확인','환경','운영','상태','그게','물','반면',
     '정보', '논리', '수준', '세기', '거리', '면', '먼저',
    '아버지', '공감', '뭐', '적극', '식', '배경', '친구', '지원',
    '우선', '자', '이용', '워', '달리', '행위', '미국인', '만큼', '분위기',
     '위원장', '위원회', '이전', '의도', '전혀', '투자', '란', '채',
    '상징', '고', '음', '명의', '유지', '조', '감정', '초', '함',
    '기', '누군가', '보기', '발생', '순간', '주류', '경찰', '앵커',
    '방향', '개발', '인터넷', '조직', '최', 
    '스', '설정', '씨', '논의', '머리', '준비', '얼마나', '새', '성과',
    '그대로', '시도', '인', '흥행', '끝', '손', '기반', '특정',
    '즉', '마찬가지', '거기', '부', '원칙', '진', '위원', '몸', '반영', '해외',
    '분노', '제목', '서비스', '전문가', '예술',
    '과도', '본인', '흐름', '대화', '그녀', '할리우드', '묘사', '글로벌',
    '주제', '요즘', '예상', '효과', '다수', '위협', '산업', '어디', '출마', '관점',
    '소개', '프로그램', '성공', '집중', '대응', '정', '기억','주의자',
    '과연', '구독', '수사', '한편', '장','불', '우리나라', '정치권', '일상', '시기', '제시',
    '활용', '국회', '영역', '기간', '구성', '임', '비교', '토론', '외교', '노래','도전', '무대',
    '기억', '커뮤니티', '피해', '당선인', '여론', '그동안', '사과',
    '공간', '억', '문학', '총장', '당원', '디', '그냥', '개념', '연합뉴스',
    '요소', '목표', '용어', '강', '계획', '지속', '단', '전망',
    '스타', '불만', '측면', '네', '늘', '최초', '로부터', '웃음', '기본', '일단',
    '절대', '티스', '맥락', '선수', '서사', '사상', '그때',
    '광고', '이익', '확산', '교사', '지난달', '시선', '국정', '남', '희망',
    '둘', '강요', '생활', '보', '차원', '추진','무슨','준','담론','사업','연구',
    '발','문','매력','지사','발전','아주','출시','현장',
    '현장', '일반', '피', '대사', '차지', '평론가', '크리스마스', '도시', '안보',
    '얼굴', '총리', '막', '조롱', '스토리', '전환', '직원', '배',
    '보이', '회의', '고려', '셈', '지층', '경선',
    '각종', '장르', '누가', '재미', '도움', '월일', '기대', '채널', '분야'
    '공연', '성장', '기관', '사고', '히어로', '최대', '대가', '불구', '터', '현', '쇼', '댓글', '시스템', '진짜', '반복', '무시',
    '제공', '지도자', '전문', '스타워즈', '만원', '선정', '행보', '수가', 
    '동의', '며', '이용자', '충격', '원인', '차례', '그룹',
    '신문', '주변', '원','제안','예정', '실사',
    '내부', '부통령', '과학', '내년', '근거', '치', '탓', '목적',
    '업계', '진실', '마련', '막말', '소재', '의회', '마치', '공유', '화제', '연설',
    '작업','스포츠', '변호사', '난', '계기', '느낌', '소리', '심지어',
    '형성', '만약', '정서', '관리', '중이', '한번', '판', '지구', '꿈','서구', '각','모델',
    '반', '아래', '소비', '주민',  '합의', '처', '싸움',
    '잘못', '창', '일종', '옹호', '상상', '포인트', '긍정','인생','원래','형태','공약','가수','주자','확장',
    '부동산','걱정', '에리얼', '그림', '얼마', '가정', '주체', '국',
    '바탕', '아무', '주로', '경향', '심','근본', '질', '수정', '탈', '항상',
    '생', '보장', '전국', '인상', '달라','주지','플로리다주',
    '소통', '파괴', '신뢰', '대의', '절차', '생명', '석',
    '의지', '외', '시청자', '검사', '확대', '탄생','공주','설득','인류','분열',
    '모바일','오마이','자연','과','디지털','친','사이트',
    '접근', '나이', '개입', '현지', '저희', '각자', '뉴욕타임스', '김현정', '기획',
    '중단', '일리', '통', '영어', '너', '낼', '엄마', '공부', '면서','사회'
])

# 7. 불용어 제거
def remove_stop(tokens):
    return [t for t in tokens if t not in domain_stopwords and len(t) > 1]

merged_df['nouns_clean'] = merged_df['nouns'].apply(remove_stop)

# 8. 결과 확인
merged_df[['제목', '본문', 'clean_text', 'nouns', 'nouns_clean']].head(10)


merged_df.to_csv('토픽_분석_결과.csv',index=False, encoding='utf-8-sig')


In [ ]:
merged_df['주토픽'].value_counts()

In [ ]:
merged_df

In [ ]:
# 1. 리스트를 문자열로 변환
corpus = merged_df['nouns_clean'].apply(lambda x: ' '.join(x))

# 2. TF-IDF 벡터화 (상위 500개 단어)
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(corpus)

# 3. 단어별 TF-IDF 점수 정렬
tfidf_scores = zip(vectorizer.get_feature_names_out(), X.sum(axis=0).tolist()[0])
sorted_tfidf = sorted(tfidf_scores, key=lambda x: x[1], reverse=False)

# 4. 결과 출력 (단어, 점수)
for word, score in sorted_tfidf[:300]:  # 상위 50개만 예시로
    print(f"{word}: {score:.4f}")




In [ ]:
additional_stopwords = {'따라서', '아마', '갑자기', '별로', '당장', '보통', '무조건', '나은', '직후', '반드시',
                        '결코', '적지', '아예', '정작', '이건', '언제', '점점', '하라', '다소', '각각', '말로',
                        '왜곡', '일보', '오늘날', '자주', '업체','구조','뮤지컬',
                        '영화', '디즈니', '게임', '드라마', '소설', '마블', '애니메이션', '트위터', '캐스팅', '가족',
                        '코로나', '페이스북', '아들', '테러', '공연', '뉴욕', '아시아', '독일', '넷플릭스', '결혼',
                        '코미디', '유튜브', '연출', '러시아', '만화', '메리', '워싱턴', '뉴시스', '외모', '부모', '협회',
                        '뮤지컬', '죽음', '브랜드', '번역', '액션', '백설공주', '동물', '소셜미디어', '업체', '백악관', '의사',
                        '소녀', '이정재', '대회', '제다이', '억원', '시즌', '칼럼', '할리', '공동체', '창작', '문장', '고통', '욕망',
                        '예능', '남녀', '평점', '경기', '아카데미', '불법', '소비자', '남편', '상처', '화가', '교육감', '질서', '연대',
                        '발견', '유명', '경향신문', '올림픽', '그린', '영웅', '시사', '힐러리', '아내', '신청', '공포', '엘리트', '시상식',
                        '주연', '조선', '혼란', '저항', '자본주의', '웹툰', '바람', '플로리다', '직업', '취임', '가짜', '아이돌', '경영', '센터',
                        '투쟁', '곳곳', '판매', '억압', '윌슨', '극장', '응답', '움직임', '성명', '소식', '기득권', '의문', '정리', '집회', '한계',
                        '외면', '적용', '인지', '바다', '피부', '신체', '자녀', '박씨', '관계자', '제품', '개월', '포기', '어머니', '취지', '추가',
                        '특파원', '수상', '오후', '플레이', '교체', '동료', '간다', '플랫폼', '스타일', '혁신', '동원', '규모', '미군', '편이', '삭제',
                        '편집', '다운', '연결', '파리', '실천', '경계', '대변인', '희생', '지능', '타인', '생산', '반감', '기능', '수용', '정상', '증가',
                        '조치', '캘리포니아', '일자리', '약속', '오브', '전개', '하루', '보편', '예측', '특징', '동영상', '위치', '보이지', '구체', '외국인',
                        '라이트', '대목', '확보', '홍콩', '훼손', '뿌리', '일각', '요청', '테슬라', '전형', '지식', '상식', '수단', '무기', '거래', '비평',
                        '우크라이나', '미투', '에너지', '세계관', '이하', '소속', '실패', '복지', '전반', '촬영', '본질', '살인', '강화', '조건', '과제',
                        '대립', '극복', '공동', '팬덤', '권위', '의무', '기자회견', '역대', '감수성', '중앙', '세운', '초반', '통제', '인수', '본격',
                        '위험', '도입', '진단', '사안', '폐지', '인구', '부인', '서도', '불평등', '소득', '비용', '대로', '경고', '여부', '비율', '작용',
                        '기후', '실현', '답변', '젊은이', '항의', '거론', '대비', '모순', '독립', '구분', '반박', '겨냥', '조지', '우주', '방문', '마스크',
                        '진정', '중시', '계급', '개선', '부자', '시점', '구호', '임명', '동맹', '부담', '대책', '통과', '연합', '자료', '블리자드', '성격',
                        '멕시코', '계층', '부여', '교과서', '소위', '심리', '형식', '인도', '의심', '격차', '혐의', '장벽', '수익', '지위', '보지', '캡틴',
                        '이처럼', '법적', '압박', '단계', '절반', '캠프', '증거', '전면', '게이머', '예전', '지도', '외부', '강제', '수행', '충돌', '대결',
                        '의제', '구축', '결론', '재판', '비상', '중도', '임기', '지점', '일베', '구도', '협상', '세금', '박재홍',
                        '인어공주', '논란', '공연', '국민', '중국', '원작', '변화', '지지', '반대', '시장', '이상', '대학', '교육', '역사', '지적',
                        '혐오', '세대', '영국', '서울', '시즌', '칼럼', '할리', '공동체', '창작', '문장', '고통', '욕망', '예능', '평점', '불법',
                        '소비자', '남편', '상처', '화가', '교육감', '질서', '연대', '발견', '유명', '경향신문', '올림픽', '그린', '영웅', '시사', '힐러리',
                        '아내', '신청', '공포', '엘리트', '시상식', '주연', '조선', '혼란', '저항', '자본주의', '웹툰', '바람', '플로리다', '직업', '취임',
                        '가짜', '아이돌', '경영', '센터', '투쟁', '곳곳', '판매', '억압', '윌슨', '극장', '응답', '움직임', '성명', '소식', '기득권', '의문',
                        '정리', '한계', '외면', '적용', '인지', '바다', '피부', '신체', '자녀', '박씨', '관계자', '제품', '개월', '포기', '어머니', '취지',
                        '추가', '특파원', '수상', '오후', '플레이', '교체', '동료', '간다', '플랫폼', '스타일', '혁신', '동원', '규모', '미군', '편이', '삭제',
                        '편집', '다운', '연결', '파리', '실천', '경계', '대변인', '희생', '지능', '타인', '생산', '반감', '기능', '수용', '정상', '증가',
                        '조치', '캘리포니아', '일자리', '약속', '오브', '전개', '하루', '보편', '예측', '특징', '동영상', '위치', '보이지', '구체', '외국인',
                        '라이트', '대목', '확보', '홍콩', '훼손', '뿌리', '일각', '요청', '테슬라', '전형', '지식', '상식', '수단', '무기', '거래', '비평',
                        '에너지', '세계관', '이하', '소속', '실패', '복지', '전반', '촬영', '본질', '살인', '강화', '조건', '과제', '대립', '극복', '공동',
                        '팬덤', '권위', '의무', '기자회견', '역대', '감수성', '중앙', '세운', '초반', '통제', '인수', '본격', '위험', '도입', '진단', '사안',
                        '폐지', '인구', '부인', '서도', '소득', '비용', '대로', '경고', '여부', '비율', '작용', '기후', '실현', '답변', '젊은이', '항의',
                        '거론', '대비', '모순', '독립', '구분', '반박', '겨냥', '조지', '우주', '방문', '마스크', '진정', '중시', '계급', '개선', '부자',
                        '시점', '구호', '임명', '동맹', '부담', '대책', '통과', '연합', '자료', '블리자드', '성격', '멕시코', '계층', '부여', '교과서', '소위',
                        '심리', '형식', '인도', '의심', '격차', '장벽', '수익', '지위', '보지', '캡틴', '이처럼', '법적', '압박', '단계', '절반', '캠프',
                        '증거', '전면', '게이머', '예전', '지도', '외부', '강제', '수행', '충돌', '대결', '의제', '구축', '결론', '재판', '비상', '중도',
                        '임기', '지점', '일베', '구도', '협상', '세금', '박재홍','그다음', '그간', '내내', '부터', '처럼', '뭔가', '어쨌든', '마음껏', '전부',
                        '때로는', '최소한', '상대로', '제일', '어제'
                       }


In [ ]:
all_stopwords = additional_stopwords

# 불용어 제거
merged_df['nouns_clean'] = merged_df['nouns_clean'].apply(
    lambda tokens: [t for t in tokens if t not in all_stopwords and len(t) > 1]
)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 리스트를 문자열로
corpus = merged_df['nouns_clean'].apply(lambda x: ' '.join(x))

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(corpus)

# TF-IDF 점수 추출 및 정렬
tfidf_scores = zip(vectorizer.get_feature_names_out(), X.sum(axis=0).tolist()[0])
sorted_tfidf = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)

# 출력
for word, score in sorted_tfidf[:1000]:
    print(f"{word}: {score:.4f}")

In [ ]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# 1) 본문 토큰 컬럼 이름 (실제 컬럼명으로 바꿔주세요)
token_col = 'nouns_clean'   

# 2) 전체 토큰 리스트
all_tokens = []
for tokens in merged_df[token_col]:
    if isinstance(tokens, list):
        all_tokens.extend(tokens)

# 3) 빈도 분석
counter = Counter(all_tokens)
freq_df = pd.DataFrame(counter.most_common(300), columns=['word', 'count'])
print(freq_df)



In [ ]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from IPython.display import display  # 주피터 환경일 경우

# 본문 토큰 컬럼명
token_col = 'nouns_clean'

# 전체 토큰 리스트 수집
all_tokens = []
for tokens in merged_df[token_col]:
    if isinstance(tokens, list):
        all_tokens.extend(tokens)

# 빈도 분석
counter = Counter(all_tokens)
freq_df = pd.DataFrame(counter.most_common(300), columns=['word', 'count'])

# 전체 출력
print(freq_df.to_string(index=False))  # 콘솔용
# display(freq_df)                     # Jupyter용

# CSV 저장
# freq_df.to_csv('word_frequency.csv', index=False)


In [ ]:
# 4) 워드클라우드
font_path = 'Library/Fonts/AppleGothic.ttf'

wc = WordCloud(
    font_path=font_path,
    width=800,
    height=400,
    background_color='white'
).generate_from_frequencies(counter)

plt.figure(figsize=(12, 6))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title('키워드 빈도분석')
plt.show()


In [ ]:
!pip install gensim pyLDAvis numpy pandas matplotlib scikit-learn


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora, models
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from tqdm import tqdm
import logging

# 로그 무시
logging.getLogger().setLevel(logging.ERROR)

# 전처리된 문서 리스트 예시 (여기에 실제 명사 문서 리스트를 넣으세요)
documents = merged_df['nouns_clean'].astype(str).tolist()  # ← 본인의 데이터프레임 열명 사용

# TF 벡터화
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)
vocab = vectorizer.get_feature_names_out()

# Gensim용 사전 및 말뭉치 생성
texts = [doc.split() for doc in documents]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# 지표 계산 함수들
def arun_metric(corpus, dictionary, texts, min_k, max_k):
    scores = []
    for k in tqdm(range(min_k, max_k+1)):
        lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=k, passes=10)
        doc_topic_distr = np.array([[tup[1] for tup in lda[doc]] for doc in corpus])
        topic_word_distr = np.array([lda.get_topic_terms(i, topn=len(dictionary)) for i in range(k)])
        topic_word_matrix = np.array([[val for _, val in topic] for topic in topic_word_distr])
        sv = np.linalg.svd(doc_topic_distr @ topic_word_matrix.T, compute_uv=False)
        score = np.sum(np.log(sv))
        scores.append(score)
    return scores

def cao_juan_metric(lda_models):
    return [np.mean(np.mean(np.dot(model.get_topics(), model.get_topics().T)) - np.eye(model.num_topics)) for model in lda_models]

def griffiths_metric(corpus, dictionary, min_k, max_k):
    scores = []
    for k in tqdm(range(min_k, max_k+1)):
        lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=k, passes=10)
        scores.append(lda.log_perplexity(corpus))
    return scores

def deveaud_metric(lda_models):
    from scipy.spatial.distance import jensenshannon
    scores = []
    for model in lda_models:
        sims = []
        for i in range(model.num_topics):
            for j in range(i+1, model.num_topics):
                p = model.get_topic_terms(i, topn=len(dictionary))
                q = model.get_topic_terms(j, topn=len(dictionary))
                p_vec = np.zeros(len(dictionary))
                q_vec = np.zeros(len(dictionary))
                for id, val in p:
                    p_vec[id] = val
                for id, val in q:
                    q_vec[id] = val
                sims.append(jensenshannon(p_vec, q_vec)**2)
        scores.append(np.mean(sims))
    return scores

# 하이퍼파라미터 범위
min_topics = 2
max_topics = 30

# LDA 모델들 생성
lda_models = [
    models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=k, passes=10, random_state=42)
    for k in tqdm(range(min_topics, max_topics+1))
]

# 각 지표 계산
cao_scores = cao_juan_metric(lda_models)
griffiths_scores = griffiths_metric(corpus, dictionary, min_topics, max_topics)
deveaud_scores = deveaud_metric(lda_models)
# Arun은 계산량이 커서 선택사항
# arun_scores = arun_metric(corpus, dictionary, texts, min_topics, max_topics)

# 그래프 시각화
plt.figure(figsize=(12, 6))
x = list(range(min_topics, max_topics+1))
plt.plot(x, cao_scores, label='CaoJuan2009', marker='o')
plt.plot(x, griffiths_scores, label='Griffiths2004', marker='o')
plt.plot(x, deveaud_scores, label='Deveaud2014', marker='o')
# plt.plot(x, arun_scores, label='Arun2010', marker='o')  # 선택적으로

plt.axvline(x=16, color='red', linestyle='--', label='Best topic: 16')
plt.legend()
plt.title('하이퍼파라미터 튜닝에 의한 모델 성능 지표')
plt.xlabel('number of topics')
plt.ylabel('score')
plt.grid(True)
plt.show()


In [ ]:
pip install gensim pandas matplotlib seaborn

In [ ]:
# 결과를 데이터프레임으로 정리
result_df = pd.DataFrame({
    'Num Topics': list(topic_range),
    'Coherence Score (c_v)': coherence_scores,
    'Perplexity Score (log)': perplexity_scores
})

# Coherence Score 기준 내림차순 정렬
result_df_sorted = result_df.sort_values(by='Coherence Score (c_v)', ascending=False)

# 전체 출력
pd.set_option('display.max_rows', None)
print(result_df_sorted.to_string(index=False))


In [ ]:
merged_df

In [ ]:
plt.figure(figsize=(12, 6))

# Coherence
plt.plot(topic_range, coherence_scores, marker='o', label='Coherence (c_v)')
# Perplexity
plt.plot(topic_range, perplexity_scores, marker='x', label='Perplexity')

plt.xlabel("Number of Topics")
plt.ylabel("Score")
plt.title("Optimal Number of Topics by Coherence and Perplexity")
plt.axvline(x=topic_range[np.argmax(coherence_scores)], color='r', linestyle='--', label=f'Best topic: {topic_range[np.argmax(coherence_scores)]}')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# pyLDAvis 시각화 준비
vis_data = gensimvis.prepare(
    topic_model=lda_model,     # 학습된 LDA 모델
    corpus=corpus,             # BOW 형식 코퍼스
    dictionary=dictionary,     # 사전
    sort_topics=False          # 토픽 정렬 유지
)

# 노트북 환경에서 시각화
pyLDAvis.display(vis_data)

# 또는 HTML로 저장
# pyLDAvis.save_html(vis_data, 'lda_vis.html')


토픽수 7로 설정하여 다시 시각화

In [ ]:
from gensim import corpora, models

# 텍스트 리스트 및 사전/코퍼스
texts = merged_df['nouns_clean'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# LDA 모델 학습
lda_7 = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=7,
    random_state=42,
    passes=10,
    iterations=100,
    alpha='auto',
    eta='auto'
)


In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# pyLDAvis 준비
vis_data = gensimvis.prepare(
    topic_model=lda_7,
    corpus=corpus,
    dictionary=dictionary,
    sort_topics=False
)

# Jupyter 환경에서 바로 시각화
pyLDAvis.display(vis_data)


토픽수 8로 설정하여 다시 시각화

In [ ]:
from gensim import corpora, models

# 텍스트 리스트 및 사전/코퍼스
texts = merged_df['nouns_clean'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# LDA 모델 학습
lda_8 = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=8,
    random_state=42,
    passes=10,
    iterations=100,
    alpha='auto',
    eta='auto'
)


In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# pyLDAvis 준비
vis_data = gensimvis.prepare(
    topic_model=lda_8,
    corpus=corpus,
    dictionary=dictionary,
    sort_topics=False
)

# Jupyter 환경에서 바로 시각화
pyLDAvis.display(vis_data)


In [ ]:
from gensim import corpora, models

# 텍스트 리스트 및 사전/코퍼스
texts = merged_df['nouns_clean'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# LDA 모델 학습
lda_3 = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=3,
    random_state=42,
    passes=10,
    iterations=100,
    alpha='auto',
    eta='auto'
)


In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# pyLDAvis 준비
vis_data = gensimvis.prepare(
    topic_model=lda_3,
    corpus=corpus,
    dictionary=dictionary,
    sort_topics=False
)

# Jupyter 환경에서 바로 시각화
pyLDAvis.display(vis_data)


In [ ]:
from gensim import corpora, models

# 텍스트 리스트 및 사전/코퍼스
texts = merged_df['nouns_clean'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# LDA 모델 학습
lda_4 = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=4,
    random_state=42,
    passes=10,
    iterations=100,
    alpha='auto',
    eta='auto'
)


In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# pyLDAvis 준비
vis_data = gensimvis.prepare(
    topic_model=lda_4,
    corpus=corpus,
    dictionary=dictionary,
    sort_topics=False
)

# Jupyter 환경에서 바로 시각화
pyLDAvis.display(vis_data)


In [ ]:
from gensim import corpora, models

# 텍스트 리스트 및 사전/코퍼스
texts = merged_df['nouns_clean'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# LDA 모델 학습
lda_4 = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5,
    random_state=42,
    passes=10,
    iterations=100,
    alpha='auto',
    eta='auto'
)


In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# pyLDAvis 준비
vis_data = gensimvis.prepare(
    topic_model=lda_4,
    corpus=corpus,
    dictionary=dictionary,
    sort_topics=False
)

# Jupyter 환경에서 바로 시각화
pyLDAvis.display(vis_data)


In [ ]:
from gensim import corpora, models

# 텍스트 리스트 및 사전/코퍼스
texts = merged_df['nouns_clean'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# LDA 모델 학습
lda_6 = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=6,
    random_state=42,
    passes=10,
    iterations=100,
    alpha='auto',
    eta='auto'
)

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# pyLDAvis 준비
vis_data = gensimvis.prepare(
    topic_model=lda_6,
    corpus=corpus,
    dictionary=dictionary,
    sort_topics=False
)

# Jupyter 환경에서 바로 시각화
pyLDAvis.display(vis_data)

In [ ]:
!pip install --upgrade tabulate

In [ ]:
from gensim import corpora, models
from collections import Counter
import pandas as pd

# 1) 말뭉치 불러오기
#    'nouns_clean' 열이 문자열 형태 리스트라면 ast.literal_eval 적용 필요
import ast
merged_df['nouns_clean'] = merged_df['nouns_clean'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
texts = merged_df['nouns_clean'].tolist()

# 2) 사전(Dictionary) 및 코퍼스(corpus) 생성
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# 3) LDA 모델 재학습 (토픽 수 = 3)
lda_3 = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=3,
    random_state=42,
    passes=10,
    iterations=100,
    alpha='auto',
    eta='auto'
)

# 4) 토픽별 주요 키워드 추출 (상위 10개)
topic_keywords = {
    i: [word for word, _ in lda_3.show_topic(i, topn=10)]
    for i in range(3)
}

# 5) 문서별 토픽 배정 (가장 높은 확률 토픽)
doc_topics = [
    max(lda_3.get_document_topics(bow), key=lambda x: x[1])[0]
    for bow in corpus
]

# 6) 토픽별 문서 수 집계
topic_counts = Counter(doc_topics)

# 7) 결과 테이블 생성
df_topics = pd.DataFrame([
    {
        '토픽 번호': i,
        '주요 키워드': ', '.join(topic_keywords[i]),
        '문서 수': topic_counts.get(i, 0)
    }
    for i in range(3)
])

# 8) 문서 수 내림차순 정렬 및 순위 컬럼 추가
df_topics = df_topics.sort_values('문서 수', ascending=False).reset_index(drop=True)
df_topics.insert(0, '순위', range(1, len(df_topics) + 1))

# 9) 출력
print(df_topics.to_string(index=False))

In [ ]:
# 각 문서별로 가장 높은 확률의 토픽 번호를 '주토픽' 열로 추가
merged_df['주토픽'] = doc_topics


In [ ]:
topic_keywords = {
    i: [word for word, _ in lda_3.show_topic(i, topn=30)]
    for i in range(3)
}

In [ ]:
topic_keywords

In [ ]:
import pandas as pd

# 토픽별 통계 및 대표 제목 추출
rows = []
for topic_id, info in topic_info.items():
    count = len(info['gammas'])
    avg_gamma = sum(info['gammas']) / count

    # γ값 기준 상위 5개 문서 추출
    top5 = sorted(info['docs'], key=lambda x: x[1], reverse=True)[:5]

    # 상위 문서의 기사 제목 가져오기
    top_titles = [merged_df.loc[i, '제목'] for i, _ in top5]

    # 테이블 형태로 저장
    rows.append({
        '토픽 번호': topic_id + 1,  # 1-based
        '문서 수': count,
        '평균 γ값': round(avg_gamma, 4),
        '대표 기사 제목(상위5)': '\n'.join(top_titles)
    })

# 데이터프레임으로 정리
df_summary = pd.DataFrame(rows)
print(df_summary.to_string(index=False))


In [ ]:
df_summary

In [ ]:
from collections import defaultdict, Counter

# γ값 임계치 설정 (0.8 이상 등)
gamma_threshold = 0.8

# 토픽별로 γ 높은 문서 인덱스를 저장할 딕셔너리
high_gamma_docs = defaultdict(list)

# 각 문서별로 γ값을 구해서, 높은 문서를 해당 토픽에 분배
for doc_idx, bow in enumerate(corpus):
    topic_probs = lda_3.get_document_topics(bow)
    for topic_id, gamma in topic_probs:
        if gamma >= gamma_threshold:
            high_gamma_docs[topic_id].append(doc_idx)

# 결과 저장용
topic_keywords_summary = []

# 각 토픽별로 주요 키워드 추출
for topic_id, doc_indices in high_gamma_docs.items():
    # 명사 리스트 결합
    topic_words = []
    for i in doc_indices:
        topic_words.extend(merged_df.loc[i, 'nouns_clean'])
    
    # 상위 키워드 20개 추출
    top_keywords = Counter(topic_words).most_common(20)
    keyword_list = [kw for kw, _ in top_keywords]

    topic_keywords_summary.append({
        '토픽 번호': topic_id + 1,  # 1-based
        '문서 수': len(doc_indices),
        '상위 키워드': ', '.join(keyword_list)
    })

# 결과 DataFrame
df_topic_keywords = pd.DataFrame(topic_keywords_summary)
print(df_topic_keywords.to_string(index=False))


In [ ]:
from collections import defaultdict

gamma_threshold = 0.8  # 필요 없으면 지워도 됨, 어차피 상위 5개 뽑으니까

# 토픽별로 (문서번호, γ값) 저장
topic_doc_gamma = defaultdict(list)

for doc_idx, bow in enumerate(corpus):
    topic_probs = lda_3.get_document_topics(bow)
    for topic_id, gamma in topic_probs:
        topic_doc_gamma[topic_id].append((doc_idx, gamma))

# 각 토픽별 상위 5개 문서 뽑기 (γ 기준 내림차순)
summary = []

for topic_id, doc_gamma_list in topic_doc_gamma.items():
    # γ 내림차순 정렬
    sorted_docs = sorted(doc_gamma_list, key=lambda x: x[1], reverse=True)
    
    # 상위 5개만
    top_5_docs = sorted_docs[:5]
    
    for doc_idx, gamma in top_5_docs:
        # 주요내용 컬럼 이름은 merged_df에서 주요 텍스트 컬럼명으로 바꿔주세요.
        main_content = merged_df.loc[doc_idx, 'text']  # 예시: text 컬럼에 주요내용 있다고 가정
        
        summary.append({
            '토픽': topic_id + 1,
            '문서번호': doc_idx,
            '주요내용': main_content,
            'γ값': round(gamma, 4)
        })

import pandas as pd

df_summary = pd.DataFrame(summary)
print(df_summary)


In [ ]:
df_summary.to_csv('df_summary.csv')

In [ ]:
print(merged_df.head())


In [ ]:
merged_df.to_csv('LDA토픽모델링',index=False, encoding='utf-8-sig')

In [ ]:
# 1) 각 토픽별 상위 5개 단어와 β값 가져오기
topic_beta = {
    topic_id: lda_3.show_topic(topic_id, topn=5)
    for topic_id in range(lda_3.num_topics)
}

# 2) 데이터프레임으로 변환
rows = []
for topic_id, terms in topic_beta.items():
    for word, beta in terms:
        rows.append({
            '토픽 번호': topic_id,
            '단어': word,
            'β 값': round(beta, 6)  # 소수점 6자리로 포맷팅
        })

df_beta = pd.DataFrame(rows)

# 3) 토픽별로 보기 좋게 정렬
df_beta = df_beta.sort_values(['토픽 번호', 'β 값'], ascending=[True, False]) \
                 .reset_index(drop=True)

# 4) 출력
print(df_beta.to_string(index=False))


In [ ]:
import pandas as pd
from collections import defaultdict

# 1) γ값(문서‑토픽 확률) 계산 (minimum_probability=0으로 모든 토픽 확률 반환)
doc_topic_dists = [
    lda_3.get_document_topics(bow, minimum_probability=0)
    for bow in corpus
]

# 2) 토픽별로 (문서 인덱스, γ값) 리스트 생성
topic_to_docs = defaultdict(list)
for doc_idx, dist in enumerate(doc_topic_dists):
    for topic_id, gamma in dist:
        topic_to_docs[topic_id].append((doc_idx, gamma))

# 3) 각 토픽별 γ 기준 상위 5개 문서 선택
top5_by_topic = {
    topic_id: sorted(docs, key=lambda x: x[1], reverse=True)[:5]
    for topic_id, docs in topic_to_docs.items()
}

# 4) 결과를 담을 DataFrame 생성
rows = []
for topic_id, docs in top5_by_topic.items():
    for doc_idx, gamma in docs:
        nouns = merged_df.loc[doc_idx, 'nouns_clean']  # 리스트 형태
        rows.append({
            '토픽 번호': topic_id,
            '문서 인덱스': doc_idx,
            'γ 값': round(gamma, 4),
            'nouns_clean': nouns
        })

df_top5_nouns = pd.DataFrame(rows)

# 5) 보기 쉽게 정렬
df_top5_nouns = df_top5_nouns.sort_values(
    ['토픽 번호', 'γ 값'],
    ascending=[True, False]
).reset_index(drop=True)

# 6) 출력
print(df_top5_nouns.to_string(index=False))


In [ ]:
import pandas as pd
from collections import defaultdict

# 0) 토픽 수 지정 (예: 16)
num_topics = lda_3.num_topics  # 이미 모델에 맞춰져 있다면 이처럼 사용 가능

# 1) 모든 문서에 대해 γ 분포 계산 (minimum_probability=0 으로 모든 토픽 확률 포함)
doc_topic_dists = [
    lda_3.get_document_topics(bow, minimum_probability=0)
    for bow in corpus
]

# 2) 토픽별 (문서 인덱스, γ값) 리스트 생성
topic_to_docs = defaultdict(list)
for doc_idx, dist in enumerate(doc_topic_dists):
    for topic_id, gamma in dist:
        topic_to_docs[topic_id].append((doc_idx, gamma))

# 3) 각 토픽별 γ 내림차순 상위 5개 문서 선택
top5_by_topic = {
    t: sorted(topic_to_docs[t], key=lambda x: x[1], reverse=True)[:5]
    for t in range(num_topics)
}

# 4) 엑셀 파일로 저장하기 위해 각 토픽별 DataFrame 생성
with pd.ExcelWriter('topic_top5_nouns.xlsx') as writer:
    for topic_id, docs in top5_by_topic.items():
        rows = []
        for doc_idx, gamma in docs:
            nouns = merged_df.loc[doc_idx, 'nouns_clean']
            rows.append({
                '토픽 번호': topic_id + 1,            # 사람 읽기 편하게 1-based
                '문서 인덱스': doc_idx,
                'γ 값': round(gamma, 4),
                'nouns_clean': ', '.join(nouns)       # 리스트 → 문자열
            })
        df_topic = pd.DataFrame(rows)
        # 시트 이름: "표7_토픽1" 등으로 자동 생성
        sheet_name = f"표{7+topic_id}_토픽{topic_id+1}"
        df_topic.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ topic_top5_nouns.xlsx 파일이 생성되었습니다.")


In [ ]:
import pandas as pd
from collections import Counter

# 1) 문서별 토픽 배정 (이미 계산해 두셨다면 이 부분은 생략 가능)
#    bow_list = corpus  # 앞서 만드신 corpus
#    doc_topics = [max(lda_3.get_document_topics(bow), key=lambda x: x[1])[0] for bow in bow_list]

# 2) 토픽별 문서 수 집계
topic_counts = Counter(doc_topics)

# 3) 전체 문서 수
total_docs = len(doc_topics)

# 4) 결과 테이블 생성
df_overall = pd.DataFrame([
    {
        '토픽 번호': topic_id + 1,                   # 사람이 보기 편하게 1-based
        '문서 수': count,
        '비율(%)': round(count / total_docs * 100, 2)
    }
    for topic_id, count in topic_counts.items()
])

# 5) 문서 수 내림차순 정렬 및 순위 컬럼 추가
df_overall = (
    df_overall
    .sort_values('문서 수', ascending=False)
    .reset_index(drop=True)
)
df_overall.insert(0, '순위', df_overall.index + 1)

# 6) 출력
print(df_overall.to_string(index=False))


In [ ]:
merged_df

In [ ]:
import matplotlib.pyplot as plt

# --- 1) 한글 폰트 설정 ---
# macOS의 경우
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

# --- 2) 스택형 막대차트 그리기 ---
fig, ax = plt.subplots(figsize=(12, 6))
# df_year_topic: index=연도, columns=토픽번호, values=평균 γ값
df_year_topic.plot(
    kind='bar',
    stacked=True,
    ax=ax
)

ax.set_xlabel('연도', fontsize=12)
ax.set_ylabel('평균 γ값 비중', fontsize=12)
ax.set_title('연도별 토픽 비중 변화 (2010–2024)', fontsize=14)
ax.legend(
    title='토픽 번호: β 상위 5개 단어',
    bbox_to_anchor=(1.05, 1),
    loc='upper left'
)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
from collections import defaultdict, Counter

# 1) 문서별 토픽 γ 분포 계산
doc_topic_dists = [
    dict(lda_3.get_document_topics(bow, minimum_probability=0))
    for bow in corpus
]

# 2) merged_df에 γ 분포 컬럼 추가
merged_df = merged_df.copy()
merged_df['gamma_dist'] = doc_topic_dists

# 3) 토픽별 정보 수집
topic_info = defaultdict(lambda: {'gammas': [], 'docs': []})

for idx, row in merged_df.iterrows():
    for topic_id, gamma in row['gamma_dist'].items():
        topic_info[topic_id]['gammas'].append(gamma)
        topic_info[topic_id]['docs'].append((idx, gamma))

# 4) 토픽별 통계 및 대표 제목 추출
rows = []
for topic_id, info in topic_info.items():
    count = len(info['gammas'])
    avg_gamma = sum(info['gammas']) / count
    # γ값 기준으로 상위 5개 문서 인덱스 뽑기
    top5 = sorted(info['docs'], key=lambda x: x[1], reverse=True)[:5]
    # 문서 제목 리스트 (컬럼명에 맞게 조정해주세요)
    top_titles = [ merged_df.loc[i, '제목'] for i, _ in top5 ]
    rows.append({
        '토픽 번호': topic_id + 1,               # 1‑based 번호
        '문서 수': count,
        '평균 γ값': round(avg_gamma, 4),
        '대표 기사 제목(상위5)': '\n'.join(top_titles)
    })

# 5) DataFrame 생성 및 정렬
df_summary = pd.DataFrame(rows)
df_summary = df_summary.sort_values('문서 수', ascending=False).reset_index(drop=True)
df_summary.insert(0, '순위', df_summary.index + 1)

# 6) 출력
print(df_summary.to_string(index=False))


In [ ]:
import pandas as pd
from collections import defaultdict

# 문서별 토픽 분포 (γ값)
doc_topic_dists = [
    dict(lda_3.get_document_topics(bow, minimum_probability=0.0))
    for bow in corpus
]

# 토픽 수
num_topics = lda_3.num_topics

# 각 토픽별 γ값 누적
topic_gamma = defaultdict(list)

# 각 문서에서 모든 토픽의 γ값 누적
for dist in doc_topic_dists:
    for topic_id in range(num_topics):
        gamma = dist.get(topic_id, 0.0)
        topic_gamma[topic_id].append(gamma)

# 평균 γ값 계산
topic_avg_gamma = {
    topic_id: round(sum(gammas) / len(gammas), 4)
    for topic_id, gammas in topic_gamma.items()
}

# 정리된 결과를 DataFrame으로 보기 좋게 출력
df_avg_gamma = pd.DataFrame([
    {"토픽 번호": topic_id + 1, "평균 토픽 소속 확률 (γ)": gamma}
    for topic_id, gamma in sorted(topic_avg_gamma.items())
])

print(df_avg_gamma.to_string(index=False))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

# 문서별 연도 정보가 들어 있는 리스트 (예: [2020, 2020, 2021, ...])
# 문서 순서와 corpus의 순서가 같아야 함
years = df_overall["연도"].tolist()  # 또는 따로 리스트로 보관된 경우 그대로 사용

# 문서별 토픽 분포 (γ값 리스트)
doc_topic_dists = [
    dict(lda_3.get_document_topics(bow, minimum_probability=0.0))
    for bow in corpus
]

# 문서별 주토픽 추출
dominant_topics = [
    max(dist.items(), key=lambda x: x[1])[0]  # γ값이 가장 큰 토픽
    for dist in doc_topic_dists
]

# 연도별 주토픽 카운트
year_topic_counts = defaultdict(lambda: defaultdict(int))
for year, topic in zip(years, dominant_topics):
    year_topic_counts[year][topic] += 1

# DataFrame으로 변환
df_topic_by_year = pd.DataFrame(year_topic_counts).fillna(0).astype(int).T
df_topic_by_year.columns = [f"Topic {i+1}" for i in df_topic_by_year.columns]

# 시각화
df_topic_by_year.plot(kind='line', figsize=(12, 6), marker='o')
plt.title("연도별 주토픽 분포")
plt.xlabel("연도")
plt.ylabel("문서 수")
plt.xticks(rotation=45)
plt.legend(title="토픽")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# 1. 문서별 γ 값 기반으로 주 토픽 추출
doc_topics = []
for i, doc_bow in enumerate(corpus):
    topic_dist = lda_model.get_document_topics(doc_bow)
    sorted_topic = sorted(topic_dist, key=lambda x: x[1], reverse=True)
    main_topic = sorted_topic[0][0]  # 가장 확률 높은 토픽
    main_prob = sorted_topic[0][1]   # 해당 확률
    doc_topics.append((main_topic, main_prob))

# 2. merged_df에 주 토픽 및 확률 열 추가
merged_df['주토픽'] = [topic for topic, prob in doc_topics]
merged_df['토픽확률'] = [prob for topic, prob in doc_topics]

# (선택) 주토픽별 기사 수 확인
topic_counts = merged_df['주토픽'].value_counts().sort_index()
print("토픽별 기사 수:\n", topic_counts)


In [ ]:
# 1. 토픽별 기사 수 계산
topic_counts = merged_df['주토픽'].value_counts().sort_index()

# 2. 200개 이상인 토픽만 필터링
topics_over_200 = topic_counts[topic_counts >= 200]

# 3. 출력
print("✅ 200개 이상 기사 보유 토픽:")
print(topics_over_200)


In [ ]:
# 1. 기사 수 기준 필터링
topic_counts = merged_df['주토픽'].value_counts().sort_index()
topics_over_200 = topic_counts[topic_counts >= 200]

# 2. 토픽 번호를 DataFrame으로 변환
df_over_200 = topics_over_200.reset_index()
df_over_200.columns = ['토픽 번호', '기사 수']

# 3. 토픽 키워드와 병합
df_topics_summary = pd.merge(df_over_200, df_topics[['토픽 번호', '주요 키워드']], on='토픽 번호', how='left')

# 4. 출력
print(df_topics_summary.to_markdown(index=False))


In [ ]:
print(df_topics.columns)

In [ ]:
df_topics

In [ ]:
import pandas as pd

# 1. 주토픽 기준으로 기사 수 세기
topic_counts = merged_df['주토픽'].value_counts().reset_index()
topic_counts.columns = ['토픽 번호', '문서 수']
topic_counts = topic_counts.sort_values('토픽 번호').reset_index(drop=True)

# 2. 각 토픽의 상위 키워드 추출 (Top 10)
topics_keywords = []
for i in range(lda_model.num_topics):
    terms = lda_model.show_topic(i, topn=10)
    keywords = ", ".join([term for term, weight in terms])
    topics_keywords.append({'토픽 번호': i, '주요 키워드': keywords})

df_keywords = pd.DataFrame(topics_keywords)

# 3. 키워드 + 문서 수를 토픽 번호 기준으로 merge
df_topics = pd.merge(topic_counts, df_keywords, on='토픽 번호', how='left')

# 4. 정렬 및 순위 부여
df_topics = df_topics.sort_values('문서 수', ascending=False).reset_index(drop=True)
df_topics.insert(0, '순위', range(1, len(df_topics) + 1))

# 5. 출력
print(df_topics[['순위', '토픽 번호', '주요 키워드', '문서 수']])


In [ ]:
df_topics

In [ ]:
doc_topics

In [ ]:
merged_df

In [ ]:
merged_df2 = merged_df[['제목','본문','nouns_clean','gamma_dist','주토픽','토픽확률']]

In [ ]:
merged_df2

In [ ]:
merged_df2.to_csv('merged2_df.csv',index=False, encoding='utf-8-sig')

In [ ]:
merged_df.to_csv('토픽_분석_결과.csv',index=False, encoding='utf-8-sig')

In [ ]:
merged_df

In [ ]:
import pandas as pd
merged_df = pd.read_csv('토픽_분석_결과.csv')

In [ ]:
from gensim import corpora, models
from collections import Counter
import pandas as pd
import ast

# 1) 말뭉치 불러오기
merged_df['nouns_clean'] = merged_df['nouns_clean'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
texts = merged_df['nouns_clean'].tolist()

# 2) 사전(Dictionary) 및 코퍼스 생성
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# 3) LDA 모델 학습 (토픽 수 = 5)
lda_5 = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5,
    random_state=42,
    passes=10,
    iterations=100,
    alpha='auto',
    eta='auto'
)

# 4) 토픽별 주요 키워드 추출 (상위 10개)
topic_keywords = {
    i: [word for word, _ in lda_5.show_topic(i, topn=10)]
    for i in range(5)  # ✅ 수정: range(5)
}

# 5) 문서별 토픽 배정
doc_topics = [
    max(lda_5.get_document_topics(bow), key=lambda x: x[1])[0]
    for bow in corpus
]

# 6) 토픽별 문서 수 집계
topic_counts = Counter(doc_topics)

# 7) 결과 테이블 생성
df_topics = pd.DataFrame([
    {
        '토픽 번호': i,
        '주요 키워드': ', '.join(topic_keywords[i]),
        '문서 수': topic_counts.get(i, 0)
    }
    for i in range(5)
])

# 8) 문서 수 기준 정렬 및 순위 부여
df_topics = df_topics.sort_values('문서 수', ascending=False).reset_index(drop=True)
df_topics.insert(0, '순위', range(1, len(df_topics) + 1))

# 9) 출력
print(df_topics.to_string(index=False))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora, models
from collections import Counter
import ast

# 1) 말뭉치 준비
merged_df['nouns_clean'] = merged_df['nouns_clean'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
texts = merged_df['nouns_clean'].tolist()

# 2) 사전 및 코퍼스 생성
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# 3) LDA 모델 학습 (토픽 수 = 3)
lda_3 = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=3,
    random_state=42,
    passes=10,
    iterations=100,
    alpha='auto',
    eta='auto'
)

# 4) 문서별 주토픽 추출
dominant_topics = [
    max(lda_3.get_document_topics(bow), key=lambda x: x[1])[0]
    for bow in corpus
]

# 5) 주토픽을 데이터프레임에 병합
merged_df['주토픽'] = dominant_topics

# 6) 연도별 주토픽 분포 집계 (비율 기준)
topic_by_year = (
    merged_df.groupby(['연도', '주토픽'])
    .size()
    .reset_index(name='count')
    .pivot(index='연도', columns='주토픽', values='count')
    .fillna(0)
)

# 비율로 변환
topic_by_year_ratio = topic_by_year.div(topic_by_year.sum(axis=1), axis=0)

# 7) 시각화 (누적 막대그래프)
topic_by_year_ratio.plot(
    kind='bar',
    stacked=True,
    figsize=(10, 6),
    colormap='tab10'
)

plt.title('연도별 주토픽 분포 비율 (LDA: 3 토픽)')
plt.ylabel('비율')
plt.xlabel('연도')
plt.legend(title='토픽 번호', loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

# 1. AppleGothic 폰트 설정
font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf'
fontprop = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=fontprop)


# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

topic_by_year_ratio.plot(
    kind='bar',
    stacked=True,
    figsize=(10, 6),
    colormap='tab10'
)

plt.title('연도별 주토픽 분포 비율 (LDA: 3 토픽)')
plt.ylabel('비율')
plt.xlabel('연도')
plt.legend(title='토픽 번호', loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
topic_by_year = (
    merged_df.groupby(['연도', '주토픽'])
    .size()
    .reset_index(name='count')
    .pivot(index='연도', columns='주토픽', values='count')
    .fillna(0)
)
topic_by_year_ratio = topic_by_year.div(topic_by_year.sum(axis=1), axis=0)

# 6. 수치만 출력
print(topic_by_year_ratio.round(3).to_string())